In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
!pip install transformers

In [30]:
import re
import os

import numpy as np
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torch.utils.data import RandomSampler
from tqdm import tqdm, trange
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer

In [31]:
PAD_TOKEN_LABEL_ID = CrossEntropyLoss().ignore_index

BATCH_SIZE = 16
LEARNING_RATE_MODEL = 1e-5
LEARNING_RATE_CLASSIFIER = 1e-3
WARMUP_STEPS = 0
GRADIENT_ACCUMULATION_STEPS = 1
MAX_GRAD_NORM = 1.0
SEED = 42
NO_CUDA = False


In [32]:
def rpad(array, n):         # pad till atleast length n
  current_len = len(array)
  if current_len > n:
    return array[:n]
  extra = n - current_len
  return array + ([0] * extra)

In [33]:
def convert_to_embedding(tokenizer, sentences_with_labels):
  for sentence, label in sentences_with_labels:
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:250]   # take only first 250
    
    to_send = tokenizer.convert_tokens_to_ids(["CLS"] + tokens + ["SEP"])
    bert_sent = rpad(to_send, n = 256)

    #Reference: https://www.geeksforgeeks.org/use-yield-keyword-instead-return-keyword-python/

    yield torch.tensor(bert_sent), torch.tensor(label, dtype = torch.int64)

In [34]:
def clean_line(line):
  line = line.strip().lower()
  line = line.replace("&nbsp;", " ")
  line = re.sub(r'<br(\s\/)?>', ' ', line)
  line = re.sub(r' +', ' ', line)  # merge multiple spaces into one

  return line

In [35]:
def read_imdb(file):
  data = []
  for line in open(file, 'r', encoding = "utf-8"):
    data.append(clean_line(line))

  return data

In [36]:
read_imdb("/content/gdrive/My Drive/BERT_sentiment/imdb_train.txt")

['bromwell high is a cartoon comedy. it ran at the same time as some other programs about school life, such as "teachers". my 35 years in the teaching profession lead me to believe that bromwell high\'s satire is much closer to reality than is "teachers". the scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools i knew and their students. when i saw the episode in which a student repeatedly tried to burn down the school, i immediately recalled ......... at .......... high. a classic line: inspector: i\'m here to sack one of your teachers. student: welcome to bromwell high. i expect that many adults of my age think that bromwell high is far fetched. what a pity that it isn\'t!',
 "if you like adult comedy cartoons, like south park, then this is nearly a similar format about the small adventures of three teenage girls at bromwell high. keisha, natella and latrin

In [37]:
def make_dataloader(tokenizer, sampler = RandomSampler, train = False):

    file = "/content/gdrive/My Drive/BERT_sentiment/imdb_train.txt" if train else "/content/gdrive/My Drive/BERT_sentiment/imdb_test.txt"

    data = read_imdb(file)

    y = np.append(np.zeros(12500), np.ones(12500))
    #Format used here is one review per line, with first 12500 lines being positive(0), 
    #followed by 12500 negative lines(1).    

    sentences_with_labels = zip(data, list(y))

    dataset = list(convert_to_embedding(tokenizer, sentences_with_labels))
    
    sampler_func = sampler(dataset) if sampler is not None else None

    dataloader = DataLoader(dataset, sampler=sampler_func, batch_size = BATCH_SIZE)

    return dataloader

In [38]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
#print(make_dataloader(tokenizer, train=True))

In [39]:
class Transformers:
  model = None

  def __init__(self, tokenizer):
    self.pad_token_label_id = PAD_TOKEN_LABEL_ID
    self.device = torch.device("cuda" if torch.cuda.is_available() and not NO_CUDA else "cpu")
    self.tokenizer = tokenizer

  def predict(self, sentence):
        if self.model is None or self.tokenizer is None:
            self.load()

        embeddings = list(convert_to_embedding([(sentence, -1)]))
        preds = self._predict_tags_batched(embeddings)
        return preds

  def evaluate(self, dataloader):
        from sklearn.metrics import classification_report
        y_pred = self._predict_tags_batched(dataloader)
        y_true = np.append(np.zeros(12500), np.ones(12500))

        score = classification_report(y_true, y_pred)
        print(score)

  def _predict_tags_batched(self, dataloader):
        preds = []
        self.model.eval()
        for batch in tqdm(dataloader, desc="Computing NER tags"):
            batch = tuple(t.to(self.device) for t in batch)

            with torch.no_grad():
                outputs = self.model(batch[0])
                _, is_neg = torch.max(outputs[0], 1)
                preds.extend(is_neg.cpu().detach().numpy())

        return preds

  def train(self, dataloader, model, epochs):
        assert self.model is None  # make sure we are not training after load() command
        model.to(self.device)
        self.model = model

        t_total = len(dataloader)    # GRADIENT_ACCUMULATION_STEPS * epochs

        # Prepare optimizer and schedule (linear warmup and decay)
        optimizer_grouped_parameters = [
            {"params": model.bert.parameters(), "lr": LEARNING_RATE_MODEL},
            {"params": model.classifier.parameters(), "lr": LEARNING_RATE_CLASSIFIER}
        ]
        optimizer = AdamW(optimizer_grouped_parameters)
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=t_total)

        # Train!
        print("***** Running training *****")
        print("Training on %d examples", len(dataloader))
        print("Num Epochs = %d", epochs)
        print("Total optimization steps = %d", t_total)

        global_step = 0
        tr_loss, logging_loss = 0.0, 0.0
        model.zero_grad()
        train_iterator = trange(epochs, desc="Epoch")
        self._set_seed()
        for _ in train_iterator:
            epoch_iterator = tqdm(dataloader, desc="Iteration")
            for step, batch in enumerate(epoch_iterator):
                model.train()
                batch = tuple(t.to(self.device) for t in batch)
                outputs = model(batch[0], labels=batch[1])
                loss = outputs[0]  # model outputs are always tuple in pytorch-transformers (see doc)

                if GRADIENT_ACCUMULATION_STEPS > 1:
                    loss = loss / GRADIENT_ACCUMULATION_STEPS

                loss.backward()

                tr_loss += loss.item()
                if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)

                    scheduler.step()  # Update learning rate schedule
                    optimizer.step()
                    model.zero_grad()
                    global_step += 1

        self.model = model

        return global_step, tr_loss / global_step

  def _set_seed(self):
    torch.manual_seed(SEED)
    if self.device == 'gpu':
      torch.cuda.manual_seed_all(SEED)

  def load(self, model_dir='weights/'):
    self.tokenizer = BertTokenizer.from_pretrained(model_dir)
    self.model = BertForSequenceClassification.from_pretrained(model_dir)
    self.model.to(self.device)



In [40]:
def train(epochs=20, output_dir="weights/"):
    num_labels = 2  # negative and positive reviews
    config = BertConfig.from_pretrained('bert-base-uncased', num_labels=num_labels)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

    dataloader = make_dataloader(tokenizer, train=True)
    predictor = Transformers(tokenizer)
    predictor.train(dataloader, model, epochs)

    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

def evaluate(model_dir="weights/"):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

    dataloader = make_dataloader(tokenizer, train=False, sampler=None)
    predictor = Transformers(tokenizer)
    predictor.load(model_dir=model_dir)
    predictor.evaluate(dataloader)


path = '/content/gdrive/My Drive/BERT_sentiment/weights/'
#os.makedirs(path, exist_ok=True)
train(epochs=10, output_dir=path)
evaluate(model_dir=path)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

***** Running training *****
Training on %d examples 1563
Num Epochs = %d 10
Total optimization steps = %d 1563


Streaming output truncated to the last 5000 lines.
Iteration:  60%|██████    | 941/1563 [12:41<08:23,  1.24it/s]


Iteration:  60%|██████    | 942/1563 [12:42<08:22,  1.24it/s]


Iteration:  60%|██████    | 943/1563 [12:43<08:21,  1.24it/s]


Iteration:  60%|██████    | 944/1563 [12:44<08:20,  1.24it/s]


Iteration:  60%|██████    | 945/1563 [12:44<08:20,  1.24it/s]


Iteration:  61%|██████    | 946/1563 [12:45<08:19,  1.23it/s]


Iteration:  61%|██████    | 947/1563 [12:46<08:18,  1.23it/s]


Iteration:  61%|██████    | 948/1563 [12:47<08:19,  1.23it/s]


Iteration:  61%|██████    | 949/1563 [12:48<08:17,  1.23it/s]


Iteration:  61%|██████    | 950/1563 [12:48<08:15,  1.24it/s]


Iteration:  61%|██████    | 951/1563 [12:49<08:15,  1.24it/s]


Iteration:  61%|██████    | 952/1563 [12:50<08:13,  1.24it/s]


Iteration:  61%|██████    | 953/1563 [12:51<08:13,  1.24it/s]


Iteration:  61%|██████    | 954/1563 [12:52<08:13,  1.24it/s]


Iteration:  61%|██████    | 955/1563 [12:52<08:11,  1

              precision    recall  f1-score   support

         0.0       0.90      0.93      0.91     12500
         1.0       0.93      0.90      0.91     12500

    accuracy                           0.91     25000
   macro avg       0.91      0.91      0.91     25000
weighted avg       0.91      0.91      0.91     25000



In [41]:
a = np.array([1,2,3])
list(a)


[1, 2, 3]

In [42]:
prll

NameError: ignored

In [ ]:
!nvidia-smi